In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!git clone https://github.com/ogunlao/saint.git

Cloning into 'saint'...
remote: Enumerating objects: 664, done.
remote: Total 664 (delta 0), reused 0 (delta 0), pack-reused 664
Receiving objects: 100% (664/664), 17.00 MiB | 22.07 MiB/s, done.
Resolving deltas: 100% (364/364), done.


In [3]:
!pip3 install -r "/content/saint/requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/14

In [74]:
!pip install torch --upgrade torch
!pip install torch --upgrade pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [73]:
import os

import numpy as np
import pandas as pd

import torch
print(torch.cuda.is_available())

True



# Setting up parameters

In [5]:
from ruamel.yaml import YAML 

**IMPORTANT : EDIT FILE-FILE INI DULU SEBELUM RUN**

Cara editnya : double-click file yg mau di edit di "Files" (tab kiri colab) ato klik link-link dibawah, terus ctrl-s buat save.

```1. /content/saint/configs/config.yaml```

```
trainer:
  max_epochs: 100 # default is 100
  # gpus: 0
  accelerator: auto
  deterministic: true
  default_root_dir: null
  # resume_from_checkpoint: null
```

```2. /content/saint/configs/data/bank_ssl.yaml```
```
data_stats:
  no_cat: 1
  no_num: 49
  cats: [1]
```

```3. /content/saint/configs/data/bank_sup.yaml```

```
data_stats:
  no_cat: 1
  no_num: 49
  cats: [1]
```

```4. /content/saint/configs/experiment/supervised.yaml```

```
experiment: supervised
task: classification # {classification, regression}
model: saint
num_output: 5 # no of output neurons: 1 for binary classification num of classes in target for multiclass}
freeze_encoder: false # freeze transformer layer
pretrained_checkpoint: null #checkpoints/lightning_logs/version_7/checkpoints/epoch=0-step=1.ckpt
```

In [6]:
config_path = 'saint/configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
  args = yaml.load(f)

print(args)

{'defaults': ['_self_', {'experiment': 'supervised'}, {'data': 'bank_sup'}], 'seed': 1234, 'transformer': {'num_layers': 6, 'num_heads': 8, 'dropout': 0.1, 'dropout_ff': 0.1, 'embed_dim': 32, 'd_ff': 32, 'cls_token_idx': 0}, 'augmentation': {'prob_cutmix': 0.3, 'alpha': 0.2, 'lambda_pt': 10}, 'optimizer': {'temperature': 0.7, 'proj_head_dim': 128, 'beta_1': 0.9, 'beta_2': 0.99, 'lr': 0.0001, 'weight_decay': 0.01, 'optim': 'adamw', 'metric': 'auroc'}, 'preproc': {'data_folder': None, 'train_split': 0.65, 'validation_split': 0.15, 'test_split': 0.2, 'num_supervised_train_data': None}, 'callback': {'monitor': 'val_loss', 'mode': 'min', 'auto_insert_metric_name': False}, 'trainer': {'max_epochs': 100, 'accelerator': 'auto', 'deterministic': True, 'default_root_dir': None}, 'dataloader': {'shuffle_val': False, 'train_bs': 32, 'val_bs': 32, 'test_bs': 16, 'num_workers': 2, 'pin_memory': False}, 'metric': '${optimizer.metric}', 'print_config': False}


# Data preprocessing

In [7]:
data_folder = "/content/saint/data"

os.mkdir("/content/saint/data")

In [8]:
dataset = pd.read_csv("/content/drive/MyDrive/SEM4/Research Method/RM Kel 19 Experiment/data-final.csv", sep='\t')

print(dataset.shape)
dataset.head()

(1015341, 110)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,2016-03-03 02:01:01,768.0,1024.0,9.0,234.0,6,1,GB,51.5448,0.1991
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,2016-03-03 02:01:20,1360.0,768.0,12.0,179.0,11,1,MY,3.1698,101.706
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,2016-03-03 02:01:56,1366.0,768.0,3.0,186.0,7,1,GB,54.9119,-1.3833
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,2016-03-03 02:02:02,1920.0,1200.0,186.0,219.0,7,1,GB,51.75,-1.25
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,2016-03-03 02:02:57,1366.0,768.0,8.0,315.0,17,2,KE,1.0,38.0


In [48]:
data = dataset.drop(list(dataset)[50:], axis=1)

print(data.shape)
data.head()

(1015341, 50)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,5.0,1.0,5.0,1.0,5.0,1.0,5.0,3.0,5.0,5.0


In [49]:
for i in data.columns:
  data = data[(data[i].notna()) & (data[i] != 0)]

print(data.shape)

(874434, 50)


In [50]:
data = data.astype(int)

In [51]:
data['EST9'].value_counts()

4    247851
2    199050
3    182001
5    133152
1    112380
Name: EST9, dtype: int64

In [52]:
data = data[:1000]

In [53]:
x = data.drop(columns=['EST9'])
y = data['EST9']

In [54]:
from saint.src.dataset import generate_splits, preprocess

In [55]:
num_supervised_train_data = 200

sup_train_indices, val_indices, test_indices, ssl_train_indices = generate_splits(len(x), num_supervised_train_data, args['preproc']['validation_split'], args['preproc']['test_split'], args['seed'],)

In [56]:
x_proc, y_proc, no_num, no_cat, cats  = preprocess(x, y, args['transformer']['cls_token_idx'])

In [57]:
print('no of numerical columns: ', no_num)
print('no of categorical columns: ', no_cat)

print('list of categories in each categorical column: ', cats)

no of numerical columns:  49
no of categorical columns:  1
list of categories in each categorical column:  [1]


In [58]:
train_df, train_y   = x_proc.iloc[sup_train_indices], y_proc.iloc[sup_train_indices]
val_df, val_y       = x_proc.iloc[val_indices], y_proc.iloc[val_indices]
test_df, test_y     = x_proc.iloc[test_indices], y_proc.iloc[test_indices]

In [59]:
train_ssl, train_ssl_y = None, None

if num_supervised_train_data != 'all':
    train_ssl, train_ssl_y = x_proc.iloc[ssl_train_indices], y_proc.iloc[ssl_train_indices]

In [60]:
train_df.to_csv('/content/saint/data/train.csv' , index=False)
train_y.to_csv('/content/saint/data/train_y.csv' , index=False)
val_df.to_csv('/content/saint/data/val.csv' , index=False)
val_y.to_csv('/content/saint/data/val_y.csv' , index=False)
test_df.to_csv('/content/saint/data/test.csv' , index=False)
test_y.to_csv('/content/saint/data/test_y.csv' , index=False)

if train_ssl is not None:
   train_ssl.to_csv('/content/saint/data/train_ssl.csv' , index=False)

if train_ssl_y is not None:
  train_ssl_y.to_csv('/content/saint/data/train_ssl_y.csv' , index=False)

# SAINT training

In [61]:
num_gpus = 1

In [62]:
os.environ["HYDRA_FULL_ERROR"] = "1"

## Self-Supervised Learning + Supervised Learning

In [63]:
# Self-Supervised Learning

!python /content/saint/main.py experiment=self-supervised \
  experiment.model=saint \
  data.data_folder=/content/saint/data \
  data=bank_ssl

/content/saint/main.py:11: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="configs", config_name="config")
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
Global seed set to 1234
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/outputs/2023-05-31/04-01-38/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type            | Params
--------------------------------------------------------
0 | transformer         | Encoder         | 65.0 K
1 | embedding        

In [80]:
# copy path file dari baris terakhir output cell di atas (hasil training self-supervised).

best_ssl_model_ckpt = "/content/outputs/2023-05-31/04-01-38/lightning_logs/version_0/checkpoints/7-120.ckpt"

edit dulu

```1. /content/saint/src/dataset.py```

```
def make_weights_for_imbalanced_classes(self):
    """adopted from https://discuss.pytorch.org/t/balanced-sampling-between-classes-with-torchvision-dataloader/2703/3"""

    nclasses = len(np.unique(self.y))
    count = np.zeros(nclasses)
    for idx in range(len(self.y)):
        target = self.y[idx]
        count[target-1] += 1

    N = float(sum(count))
    weight_per_class = N / count
    weight = np.zeros(len(self))
    for idx in range(len(self.y)):
        target = self.y[idx]
        weight[idx] = weight_per_class[target-1]
    return weight
```
```2. /content/saint/utils/utils.py```
```
def auroc(self):
  return AUROC(num_classes=self.num_classes, task='multiclass')
```
```3. /content/saint/src/trainer.py, line 81```
```
def on_training_epoch_end(self, training_step_outputs):
    self.log(f'train_{self.metric}_epoch', self.train_metric.compute(), prog_bar=True,)

    # reset after each epoch
    self.train_metric.reset()
```
```Reference : https://github.com/Lightning-AI/lightning/pull/16520```

```4. /content/saint/src/trainer.py, line 94```
```
def on_validation_epoch_end(self, validation_step_outputs):
    self.log(f'val_{self.metric}_epoch', self.valid_metric.compute(), prog_bar=True,)

    # reset after each epoch
    self.valid_metric.reset()
```
```Reference : https://github.com/Lightning-AI/lightning/pull/16520```

In [76]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [77]:
# SUP from SSL

!python /content/saint/main.py experiment=supervised \
  experiment.model=saint \
  data.data_folder=/content/saint/data \
  data=bank_sup \
  experiment.pretrained_checkpoint={best_ssl_model_ckpt}

/content/saint/main.py:11: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="configs", config_name="config")
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
Global seed set to 1234
Initializing supervised task using pretrained model:
/content/outputs/2023-05-31/04-01-38/lightning_logs/version_0/checkpoints/7-120.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/outputs/2023-05-31/04-51-29/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Para

## Supervised Learning

In [ ]:
!python /content/saint/main.py experiment=supervised \
  experiment.model=saint \
  data.data_folder=/content/saint/data \
  data=bank_sup

## Model prediction

In [ ]:
pretrained_checkpoint = "/content/outputs/2021-11-01/13-30-49/lightning_logs/version_0/checkpoints/2-20.ckpt"

In [ ]:
!python /content/saint/predict.py experiment=predict \
  experiment.model=saint \
  data=bank_sup \
  data.data_folder=/content/saint/data \
  experiment.pretrained_checkpoint={pretrained_checkpoint} \
  experiment.pred_sav_path=/content/predict.csv